In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_json("/kaggle/input/sarcasm-detection-through-nlp/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
df.to_csv('sarcasm_csv', index = False)
df = pd.read_csv("/kaggle/working/sarcasm_csv")
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
df.isnull().sum()

article_link    0
headline        0
is_sarcastic    0
dtype: int64

In [5]:
y = df.is_sarcastic
x = df.drop('is_sarcastic', axis = 1)

In [6]:
print(x.shape)
print(y.shape)

(26709, 2)
(26709,)


In [7]:
import tensorflow as tf

2024-04-24 22:06:15.453566: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 22:06:15.453671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 22:06:15.637643: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
tf.__version__

'2.15.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import RMSprop

In [10]:
voca_size = 10000

In [11]:
text = x.copy()

In [12]:
text['headline'][1]

"the 'roseanne' revival catches up to our thorny political mood, for better and worse"

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(text)):
    review = re.sub('[^a-zA-Z]', ' ', text['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['former versac store clerk sue secret black code minor shopper',
 'roseann reviv catch thorni polit mood better wors',
 'mom start fear son web seri closest thing grandchild',
 'boehner want wife listen come altern debt reduct idea',
 'j k rowl wish snape happi birthday magic way',
 'advanc world women',
 'fascin case eat lab grown meat',
 'ceo send kid school work compani',
 'top snake handler leav sink huckabe campaign',
 'friday morn email insid trump presser age',
 'airlin passeng tackl man rush cockpit bomb threat',
 'facebook reportedli work healthcar featur app',
 'north korea prais trump urg us voter reject dull hillari',
 'actual cnn jeffrey lord indefens',
 'barcelona hold huge protest support refuge',
 'nuclear bomb deton rehears spider man music',
 'cosbi lawyer ask accus come forward smear legal team year ago',
 'stock analyst confus frighten boar market',
 'bloomberg program build better citi got bigger',
 'craig hick indict',
 'courtroom sketch artist clear manga influe

In [17]:
onehot_repr=[one_hot(words,voca_size)for words in corpus]
onehot_repr

[[7032, 7744, 9766, 8924, 8421, 3585, 4792, 8527, 6247, 2635],
 [8309, 4919, 3441, 8726, 8063, 546, 9195, 4460],
 [8147, 2435, 9123, 2930, 8967, 3804, 7906, 3582, 2138],
 [998, 3288, 5801, 7598, 340, 8639, 6363, 2733, 9727],
 [8240, 5284, 9066, 4867, 1637, 8362, 8126, 8783, 5170],
 [6224, 5976, 8696],
 [9999, 3666, 6986, 8132, 1444, 6802],
 [3874, 9223, 4975, 4356, 3998, 7370],
 [5859, 8202, 597, 3700, 351, 6881, 8838],
 [6793, 2562, 5575, 7073, 6505, 3040, 5454],
 [3063, 9232, 8899, 4857, 2754, 6643, 3001, 7340],
 [6178, 5151, 3998, 7516, 4646, 5114],
 [8077, 3597, 3340, 6505, 1337, 1170, 5479, 4913, 4539, 7538],
 [8714, 6455, 1392, 9832, 7956],
 [3314, 2557, 5371, 8698, 4650, 6156],
 [3825, 3001, 9422, 1444, 6968, 4857, 6934],
 [7696, 6523, 5290, 9850, 340, 554, 2823, 4540, 3151, 1666, 1489],
 [6481, 5507, 2994, 8761, 2816, 6936],
 [7160, 878, 5152, 9195, 6044, 3805, 6651],
 [9889, 3945, 8717],
 [735, 619, 7586, 1273, 5643, 8051],
 [6505, 9777, 1080, 749, 876, 695, 706, 2460, 9499, 6

In [18]:
print(corpus[1])
print(onehot_repr[1])

roseann reviv catch thorni polit mood better wors
[8309, 4919, 3441, 8726, 8063, 546, 9195, 4460]


In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[7032 7744 9766 ...    0    0    0]
 [8309 4919 3441 ...    0    0    0]
 [8147 2435 9123 ...    0    0    0]
 ...
 [8345 2297    0 ...    0    0    0]
 [1154 1823 1958 ...    0    0    0]
 [ 444 2938 5329 ...    0    0    0]]


In [20]:
print(embedded_docs[1])

[8309 4919 3441 8726 8063  546 9195 4460    0    0    0    0    0    0
    0    0    0    0    0    0]


In [21]:
embedding_vector_features = 50  
model = Sequential()
model.add(Embedding(voca_size, embedding_vector_features))
model.add(Bidirectional(LSTM(128, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(64, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(32)))  
model.add(Dropout(0.2))  
model.add(Dense(1, activation='sigmoid'))

optimizer_rmsprop = RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer_rmsprop, metrics=['accuracy'])

In [22]:
len(embedded_docs),y.shape

(26709, (26709,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((26709, 20), (26709,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=18,batch_size=64)

Epoch 1/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5918 - loss: 0.6566 - val_accuracy: 0.7420 - val_loss: 0.5417
Epoch 2/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7963 - loss: 0.4458 - val_accuracy: 0.7784 - val_loss: 0.4719
Epoch 3/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8280 - loss: 0.3940 - val_accuracy: 0.7785 - val_loss: 0.4654
Epoch 4/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8454 - loss: 0.3641 - val_accuracy: 0.7852 - val_loss: 0.5200
Epoch 5/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8721 - loss: 0.3161 - val_accuracy: 0.7850 - val_loss: 0.4616
Epoch 6/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8829 - loss: 0.2889 - val_accuracy: 0.7767 - val_loss: 0.4955
Epoch 7/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9033 - loss: 0.2585 - val_accuracy: 0.7679 - val_loss: 0.5934
Epoch 8/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9069 - loss: 0.2455 - val_acc

In [27]:
y_pred=model.predict(X_test)

276/276 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [28]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(y_test,y_pred)

array([[3967,  937],
       [1219, 2691]])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7553891536192421

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79      4904
           1       0.74      0.69      0.71      3910

    accuracy                           0.76      8814
   macro avg       0.75      0.75      0.75      8814
weighted avg       0.75      0.76      0.75      8814



# Now with transformer


In [33]:
!pip install transformers

In [34]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Load the tokenizer and model from PyTorch weights
tokenizer = AutoTokenizer.from_pretrained("jkhan447/sarcasm-detection-Bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("jkhan447/sarcasm-detection-Bert-base-uncased", from_pt=True)


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequ

In [ ]:
import tensorflow as tf

# Compile the model (with optimizer instantiation within compile)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=18,batch_size=64)




Epoch 1/18
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
